In [1]:
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

from azureml.core import Environment, Model, Workspace
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

load_dotenv()
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP")
AZURE_WORKSPACE_NAME = os.getenv("AZURE_WORKSPACE_NAME")


version = str(uuid.uuid4())[:4]

source_directory = "./azureml"
experiment_name = "oc-p8-experiment-1"

model_names = [
    "deeplab_v3plus_64",
    "deeplab_v3plus_80",
    "deeplab_v3plus_128",
    "deeplab_v3plus_160",
    "deeplab_v3plus_256",
    "deeplab_v3plus_320",
    "deeplab_v3plus_512",
    "deeplab_v3plus_640",
]


# connect to your workspace
ws = Workspace(
    subscription_id=AZURE_SUBSCRIPTION_ID,
    resource_group=AZURE_RESOURCE_GROUP,
    workspace_name=AZURE_WORKSPACE_NAME,
)


env_name = experiment_name + "-predict"
try:
    env = Environment.get(workspace=ws, name=env_name)
except:
    env = Environment.from_conda_specification(
        name=env_name,
        file_path=Path(source_directory, "conda_dependencies.yml"),
    )
    env.inferencing_stack_version = "latest"
    env.register(workspace=ws)


inference_config = InferenceConfig(
    source_directory=source_directory,
    entry_script="predict.py",
    environment=env,
)


for model_name in model_names:
    aci_config = AciWebservice.deploy_configuration(
        cpu_cores=3.8,
        memory_gb=15,
        auth_enabled=True,
    )

    model = Model(ws, model_name)

    service = Model.deploy(
        workspace=ws,
        name=model_name.replace("_", "-") + "-" + version,
        models=[model],
        inference_config=inference_config,
        deployment_config=aci_config,
        overwrite=True,
        show_output=True,
    )

    # service.wait_for_deployment(show_output=True)


Models: ['deeplab_v3plus_64:2']
Entry script: azureml/predict.py
Environment variables: {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE', 'AZUREML_SOURCE_DIRECTORY': 'azureml', 'AZUREML_ENTRY_SCRIPT': 'azureml/predict.py'}
Environment dependencies: ['python=3.8', {'pip': ['tensorflow==2.7.0', 'azureml-defaults', 'azureml-sdk', 'inference-schema', 'numpy', 'matplotlib', 'Pillow']}]
Environment docker image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220208.v1
CPU requirement: 3.8, Memory requirement: 15GB
Uploading dependency /tmp/tmp87yahppl/e01de56c.tar.gz.
Request submitted, please run wait_for_deployment(show_output=True) to get deployment status.
Models: ['deeplab_v3plus_80:1']
Entry script: azureml/predict.py
Environment variables: {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE', 'AZUREML_SOURCE_DIRECTORY': 'azureml', 'AZUREML_ENTRY_SCRIPT': 'azureml/predict.py'}
Environment dependencies: ['python=3.8', {'pip': ['tensorflow==2.7.0', 'azureml-defaults', 'azureml-sdk', 'inference-schema', 'numpy',